In [59]:
import pandas as pd
from pulp import *
import numpy as np

In [60]:
free_agents=pd.read_csv("free_agents.csv")
raptors=pd.read_csv("raptors.csv")
raptors.rename(columns={'name': 'Player','free agent': 'raptors free agent'},inplace=True, errors='raise')
raptors['Raptors']=True
raptors= raptors[raptors['Player'].isin(free_agents['Player'])==False].reset_index(drop=True)


In [61]:
#remove anyone who has not played over 30 minutes

In [62]:
free_agents=free_agents.append(raptors).reset_index(drop=True)

In [63]:
free_agents=free_agents[free_agents['MP'] >= 100].reset_index(drop=True)


In [64]:
set(free_agents.columns)-set(raptors.columns)
#For shooting you want to evaluate the shooters not the PF right?

{'AVG SALARY', 'TYPE', 'Tm'}

Note we don't have position of the raptors players

In [65]:
BUDGET=109.14 *10**6

In [66]:
#free_agents=free_agents[free_agents['Player'].isin(raptors['name'])==False].reset_index(drop=True)

In [67]:
# Treat like portfolio optimization problem, where risk is turnover.
# Offensive team vs Defensive team
# Convex model using gorubi
# Include injury rate

In [68]:
#2020 salary was known in 2019 (assumption)
#decision vars for every player.

### Turn non-percentage values to percentages

In [69]:
percentages=free_agents.columns[free_agents.columns.str.contains("%")]
res=[free_agents[percentage]/100 if len(free_agents[free_agents[percentage]>1])>1 else free_agents[percentage] for percentage in percentages]

In [70]:
res=pd.DataFrame(res).T
for col in list(res.columns):
    free_agents[col]=res[col]

In [71]:
free_agents['eFG']= (free_agents['FG'] + 0.5 * free_agents['3P'])/ free_agents['FGA']
free_agents['TOV_Factor']=free_agents['TOV'] / (free_agents['FGA']+ 0.44 * free_agents['FTA']+ free_agents['TOV']) 
free_agents = free_agents.replace(np.nan, 0)
#free_agents['Offensive Rebounding']=free_agents['ORB'] / (free_agents['Opp ORB'] + free_agents['DRB'])
#free_agents['Defensive Rebounding']=free_agents['DRB'] / (Opp ORB + DRB)

### Decision Variables 
Note because we do not have opp ORB and DRB I am using ORB% and DRB %

In [72]:
typeplayer= list(set(free_agents['TYPE']))
positions= list(set(free_agents['Pos']))

playersall = LpVariable.dicts(name='Agent', indexs=range(len(free_agents)), cat=LpBinary) 
playertypes= LpVariable.dicts(name="Player",indexs=(range(15),positions),cat=LpBinary) # Ensure that player has 
nonfree=free_agents[(free_agents['raptors free agent']==0)&(free_agents['Raptors']==True)]
raptornonfreeagentsold=LpVariable.dicts(name='Raptors_NonAgent', indexs=nonfree.index, cat=LpBinary) 
positions = LpVariable.dicts(name='Position', indexs=positions, cat=LpBinary) # Ensure all positions are filled


In [73]:
eFG=lpSum(np.array(free_agents['eFG'])*np.array(list(playersall.values())))
TOV_Factor=lpSum(np.array(free_agents['TOV_Factor'])*np.array(list(playersall.values())))
ORB=lpSum(np.array(free_agents['ORB%'])*np.array(list(playersall.values())))
DRB=lpSum(np.array(free_agents['DRB%'])*np.array(list(playersall.values())))

playersbought=15



### Objective
Note because we cannot do non-linear objectives, we always have to have a team size of 15

In [74]:
prob = LpProblem("NBA")
prob.sense = LpMaximize



#divide by playersbought for gurobi instead of 15?
objective=lpSum(free_agents['WS']*np.array(list(playersall.values())))
secondaryobjective=lpSum((.4*eFG)/playersbought - (.25*TOV_Factor)/playersbought + (.2*ORB)/playersbought + (.15*DRB)/playersbought)
prob+=objective
prob.addConstraint(LpConstraint(name="Secondary Objective",e=secondaryobjective,sense=1,rhs=.2))







### Constraints

budget constraint: Note that some player's avg salaries are missing. Ie. Zaza has been playing for years but doesn't have an average salary. Replaced with a minimum salary of $899831

In [75]:

#ensure that we deduct the salary of non free agents on raptors if we choose to kick them
for i in raptornonfreeagentsold.keys():
    inverse=LpConstraint(name="RaptorsnonAgent_sold"+str(i),e=-1+playersall[i]+raptornonfreeagentsold[i],sense=0,rhs=0) #Constraint that if we keep 1, then this variable must be the inverse 0. Ie no matter what we pay their salary
    prob.addConstraint(inverse)

In [76]:
#budget constraint: 
averagesalary = [int(salary) for salary in free_agents['AVG SALARY'] if salary != "-"]
#Need to fill in NAs
#out of the league if no average salary
minimumsalary = 898310 #[898310,1445697,1620564,1678854,1737145,1882867,2028594,2174318,2320044,2331593,2564753]
free_agents['AVG SALARY'] = [int(salary) if salary != "-" else minimumsalary for salary in free_agents['AVG SALARY']]
##notefinished

transactioncost=0

#transactioncost=0
# ensure that raptors players that aren't free agents, if kicked, we have to pay their salary
for indexrap in raptornonfreeagentsold.keys():
    transactioncost += raptornonfreeagentsold[indexrap]*free_agents['salary_2020'][indexrap]

spending=lpSum(np.array(free_agents['AVG SALARY'])*np.array(list(playersall.values()))+transactioncost)


budgetconstraint=LpConstraint(name="budget_constraint",e=spending,sense=-1,rhs=BUDGET)
prob.addConstraint(budgetconstraint)


team constraints must be size 15

In [77]:
teamconstraint=LpConstraint(name="team_size",e=lpSum(np.array(list(playersall.values()))),rhs=15,sense=0)
prob.addConstraint(teamconstraint)

position constraints, 5 guards, 5 forwards and 2 centres

In [78]:
def positionalconstraints(df,keyword="C",cname="Center Constraint",constr=2,sense=1):
    centres=df[df['Pos'].str.contains(keyword)]
    subcentres=set(centres['Pos'])
    #does a forward Centre count as two?
    centerconstraint=0
    for center in centres.index:
        centerconstraint+=playersall[center]
    centreconstraint=LpConstraint(name=cname,e=centerconstraint,sense=sense,rhs=constr)
    return(centreconstraint)
prob.addConstraint(positionalconstraints(free_agents))
prob.addConstraint(positionalconstraints(free_agents,keyword="G",cname="Guard Constraint",constr=5))
prob.addConstraint(positionalconstraints(free_agents,keyword="F",cname="Forward Constraint",constr=5))

prob.addConstraint(positionalconstraints(free_agents,cname="Center Constraint Max",sense=-1,constr=4))
prob.addConstraint(positionalconstraints(free_agents,keyword="G",cname="Guard Constraint Max",constr=10,sense=-1))
prob.addConstraint(positionalconstraints(free_agents,keyword="F",cname="Forward Constraint Max",constr=10,sense=-1))



In [79]:
transactioncost

2351839.0*Raptors_NonAgent_192 + 33333333.0*Raptors_NonAgent_193 + 23271605.0*Raptors_NonAgent_194 + 9346153.0*Raptors_NonAgent_195 + 2281800.0*Raptors_NonAgent_196 + 10116576.0*Raptors_NonAgent_197 + 25595700.0*Raptors_NonAgent_198 + 1588231.0*Raptors_NonAgent_199 + 1000000.0*Raptors_NonAgent_200 + 0.0

In [80]:
positions

{'PF': Position_PF,
 'SF-SG': Position_SF_SG,
 'PF-SF': Position_PF_SF,
 'PG': Position_PG,
 'PG-SG': Position_PG_SG,
 'SF': Position_SF,
 'C-PF': Position_C_PF,
 'SG': Position_SG,
 'SG-SF': Position_SG_SF,
 'SG-PF': Position_SG_PF,
 'PF-C': Position_PF_C,
 'C': Position_C}

In [81]:
prob.solve()

1

In [82]:
def showresults(prob):
    # The status of the solution is printed to the screen
    print("="*30,"\nSolution Status:", LpStatus[prob.status])

    # Results
    obj = value(prob.objective)
    print("The objectve is {}".format(round(obj,2)))

    print("Optimal Solution:")
    for v in prob.variables():
        if(v.varValue==1):
            print(v.name, "=", v.varValue)

    print("Constraints")
    for c in list(prob.constraints.values()):
        print(c.name, "=", c.value())
def sensitivity(prob):
    #Print reduced costs
    o = [{'Variables':v.name,
          'Final Value':v.varValue,
          'Reduced Cost':v.dj,
          }
         for v in prob.variables()]
    print(pd.DataFrame(o),'\n')

    #Print shadow price and constraint slack
    o = [{'Constraint':name,
          'Shadow Price':c.pi,
          'RHS':-c.constant,
          'Slack': c.slack}
         for name, c in prob.constraints.items()]
    print(pd.DataFrame(o))
    
showresults(prob)

Solution Status: Optimal
The objectve is 80.7
Optimal Solution:
Agent_157 = 1.0
Agent_161 = 1.0
Agent_192 = 1.0
Agent_193 = 1.0
Agent_194 = 1.0
Agent_195 = 1.0
Agent_196 = 1.0
Agent_197 = 1.0
Agent_198 = 1.0
Agent_199 = 1.0
Agent_2 = 1.0
Agent_200 = 1.0
Agent_21 = 1.0
Agent_81 = 1.0
Agent_86 = 1.0
Constraints
Secondary_Objective = 0.019906940829984836
RaptorsnonAgent_sold192 = 0.0
RaptorsnonAgent_sold193 = 0.0
RaptorsnonAgent_sold194 = 0.0
RaptorsnonAgent_sold195 = 0.0
RaptorsnonAgent_sold196 = 0.0
RaptorsnonAgent_sold197 = 0.0
RaptorsnonAgent_sold198 = 0.0
RaptorsnonAgent_sold199 = 0.0
RaptorsnonAgent_sold200 = 0.0
budget_constraint = -1021745.0
team_size = 0.0
Center_Constraint = 2.0
Guard_Constraint = 1.0
Forward_Constraint = 2.0
Center_Constraint_Max = 0.0
Guard_Constraint_Max = -4.0
Forward_Constraint_Max = -3.0


In [83]:
sensitivity(prob)

                Variables  Final Value  Reduced Cost
0                 Agent_0          0.0           2.3
1                 Agent_1          0.0           1.5
2                Agent_10          0.0           0.6
3               Agent_100          0.0           0.3
4               Agent_101          0.0           2.2
..                    ...          ...           ...
205  Raptors_NonAgent_196          0.0           0.0
206  Raptors_NonAgent_197          0.0           0.0
207  Raptors_NonAgent_198          0.0           0.0
208  Raptors_NonAgent_199          0.0           0.0
209  Raptors_NonAgent_200          0.0           0.0

[210 rows x 3 columns] 

                 Constraint  Shadow Price          RHS         Slack
0       Secondary_Objective          -0.0          0.2 -1.990694e-02
1   RaptorsnonAgent_sold192          -0.0          1.0 -0.000000e+00
2   RaptorsnonAgent_sold193          -0.0          1.0 -0.000000e+00
3   RaptorsnonAgent_sold194          -0.0          1.0 -0.0000

### Chosen Players

In [84]:
chosen=[]
for var in prob.variables():
    if("Agent" in var.name and "Raptor" not in var.name):
        if(var.varValue==1):
            no=int(var.name.split("_")[1])
            chosen.append(free_agents['Player'][no])

            

In [85]:
free_agents[free_agents['Player'].isin(chosen)].to_csv("results.csv")
# is there a chance, if you sell a non free agent we recoup 50% cost?
free_agents[free_agents['Player'].isin(chosen)]

,Unnamed: 0,Player,TYPE,AVG SALARY,Pos,Age,Tm,G,MP,PER,...,PF,PTS,ORtg,DRtg,Rk_y,raptors free agent,salary_2020,Raptors,eFG,TOV_Factor
2,4,Willie Cauley-Stein,UFA,2231920,C,25,SAC,81,2213,18.0,...,4.8,20.3,118.0,108,0.0,0.0,0.0,0,0.557692,0.091222
21,30,Thaddeus Young,UFA,14545000,PF,30,IND,81,2489,16.2,...,3.8,20.1,113.0,106,0.0,0.0,0.0,0,0.557576,0.119474
81,94,Kevin Durant,UFA,41063925,SF,30,GSW,78,2702,24.2,...,2.7,35.7,121.0,110,0.0,0.0,0.0,0,0.569672,0.123778
86,99,Kawhi Leonard,UFA,34379100,SF,27,TOR,60,2040,25.8,...,2.0,37.5,119.0,105,0.0,0.0,0.0,0,0.547170,0.083086
157,180,Darren Collison,UFA,898310,PG,31,IND,76,2143,16.7,...,3.1,19.5,118.0,108,0.0,0.0,0.0,0,0.526667,0.146524
161,184,Danny Green,UFA,15000000,SG,31,TOR,80,2216,13.0,...,3.7,17.8,119.0,108,0.0,0.0,0.0,0,0.621324,0.102302
192,0,Pascal Siakam,0,0,PF,24,0,80,2548,18.7,...,4.5,25.5,120.0,107,1.0,0.0,2351839.0,True,0.592697,0.124957
193,2,Kyle Lowry,0,0,PG,32,0,65,2213,16.5,...,3.6,20.1,115.0,108,3.0,0.0,33333333.0,True,0.518750,0.179327
194,4,Serge Ibaka,0,0,PF-C,29,0,74,2010,18.7,...,5.0,26.5,113.0,105,5.0,0.0,23271605.0,True,0.559809,0.106099
195,5,Fred VanVleet,0,0,PG-SG,24,0,64,1760,13.7,...,3.0,19.1,114.0,110,6.0,0.0,9346153.0,True,0.503067,0.110753


Some further constraints and formulatons may include if we know the salary before or after? 

1. The salary has uncertainty, we decide on the roster, but the salary could fluctuate between one standard deviation higher with a 50% chance. So the amount paid depends on the asking price of the player. Ie. pay_xi_m = 10000,pay_xi_l = 25000 -> in the budget constraint (.5 * 10000 + .5 * 25000)*xi.
2. Relative Youth: The youth of a player is something we need to consider. If not much data is on them, a TOV of 0 is seen as good but it really isn't

### Salaries

In [86]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(averagesalary)
fig.show()

In [87]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(free_agents['MP'],nbins=100)
fig.show()